# EDA for Station Metadata

**Load**

In [1]:
import pickle

In [45]:
with open('./data/loaded/meta.dat', 'rb') as f:
    meta = pickle.load(f)
meta

,ID,Fwy,Dir,Latitude,Longitude,Length,Type,Lanes
0,715898,5,S,33.880183,-118.021787,0.430,ML,3
1,715900,5,S,33.882892,-118.026822,NaN,OR,1
2,715901,5,N,33.883400,-118.027451,NaN,OR,1
3,715903,5,N,33.886992,-118.034125,NaN,OR,1
4,715904,5,S,33.892489,-118.044573,NaN,OR,1
...,...,...,...,...,...,...,...,...
4878,777066,405,S,34.161152,-118.469631,0.455,ML,3
4879,777067,405,N,34.161136,-118.469426,0.790,HV,1
4880,777068,405,S,34.161152,-118.469631,0.790,HV,1
4881,777168,10,E,34.068993,-118.149756,0.450,HV,1


**Build Map Visualizing all Stations and General Statistics**

In [7]:
from folium import Map
import folium.plugins as plugins

In [79]:
dir_mapper = {'S': 'south', 'N': 'north', 'E': 'east', 'W': 'west'}
m = Map(location=(34.0522,-118.2437), tiles='https://tile.jawg.io/jawg-dark/{z}/{x}/{y}{r}.png?access-token=yxQukjQJyY3mRrF6htcGR22i1QJ6BP6wslSe2Cmq2k4aT8S0wbDtYMEaPhc8s240', attr='<a href="http://jawg.io" title="Tiles Courtesy of Jawg Maps" target="_blank">&copy; <b>Jawg</b>Maps</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', control_scale=True, max_bounds=True, prefer_canvas=True, zoom_start=10)
for ind, row in meta.iterrows():
    icon = folium.features.CustomIcon(f"./data/icons/{(dir_mapper[row['Dir']])}.png", icon_size=(15, 15))
    style = 'font-family: Impact, Haettenschweiler, "Franklin Gothic Bold", Charcoal, "Helvetica Inserat", "Bitstream Vera Sans Bold", "Arial Black", "sans serif";'
    html = f'''<body style="{style}"><div align="center", style="background-color: #BAD6FF; font-family: Arial">''' + row.to_frame().transpose()[['ID', 'Fwy', 'Length', 'Type', 'Lanes']].transpose().to_html(justify="center", header=False, index=True, index_names=False, col_space=300, classes="table-condensed table-responsive table-success") + '</div></body>' 
    popup = folium.Popup(html, max_width=300)                
    folium.Marker(row[['Latitude', 'Longitude']].values.tolist(), icon=icon, popup=popup).add_to(m)

In [81]:
m.save('./plots/station_map.html')

**Downsample Stations Using RDP**

In [93]:
from simplification.cutil import (
    simplify_coords,
    simplify_coords_idx,
    simplify_coords_vw,
    simplify_coords_vw_idx,
    simplify_coords_vwp,
)
import pandas as pd

In [98]:
ds_coords = pd.DataFrame(simplify_coords(meta[["Latitude", 'Longitude']].values, 0.001), columns=['Latitude', 'Longitude'])
ds_coords

,Latitude,Longitude
0,33.880183,-118.021787
1,33.900669,-118.059422
2,33.916643,-118.079557
3,33.938544,-118.094941
4,33.956897,-118.110532
...,...,...
1943,33.873704,-118.219282
1944,34.024458,-118.239564
1945,34.034315,-118.266848
1946,34.161152,-118.469631


**Downsample Stations**

All Stations

In [ ]:
m = Map(location=(34.0522,-118.2437), tiles='https://tile.jawg.io/jawg-dark/{z}/{x}/{y}{r}.png?access-token=yxQukjQJyY3mRrF6htcGR22i1QJ6BP6wslSe2Cmq2k4aT8S0wbDtYMEaPhc8s240', attr='<a href="http://jawg.io" title="Tiles Courtesy of Jawg Maps" target="_blank">&copy; <b>Jawg</b>Maps</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', control_scale=True, max_bounds=True, prefer_canvas=True, zoom_start=10)
for ind, row in meta[meta['Dir'] == 'N'].iterrows():
    icon = folium.features.CustomIcon(f"./data/icons/north.png", icon_size=(15, 15))
    style = 'font-family: Impact, Haettenschweiler, "Franklin Gothic Bold", Charcoal, "Helvetica Inserat", "Bitstream Vera Sans Bold", "Arial Black", "sans serif";'
    html = f'''<body style="{style}"><div align="center", style="background-color: #BAD6FF; font-family: Arial">''' + row.to_frame().transpose()[['ID', 'Fwy', 'Length', 'Type', 'Lanes']].transpose().to_html(justify="center", header=False, index=True, index_names=False, col_space=300, classes="table-condensed table-responsive table-success") + '</div></body>' 
    popup = folium.Popup(html, max_width=300)                
    folium.Marker(row[['Latitude', 'Longitude']].values.tolist(), icon=icon, popup=popup).add_to(m)
m

After Downsampling

In [135]:
# 1 deg lat = 69 mi
# 1 deg long = 54 mi
distance_between_stations = meta[meta['Dir'] == 'N'].sort_values(['Latitude', 'Longitude'])[['Latitude', 'Longitude']].diff()
stations = distance_between_stations[((distance_between_stations['Latitude'] * 69) > 0.1) | ((distance_between_stations['Longitude'] * 54) > 0.1)].index.tolist()
#stations = set(stations).union(set([s - 1 for s in stations]))
len(stations)

333

In [ ]:
m = Map(location=(34.0522,-118.2437), tiles='https://tile.jawg.io/jawg-dark/{z}/{x}/{y}{r}.png?access-token=yxQukjQJyY3mRrF6htcGR22i1QJ6BP6wslSe2Cmq2k4aT8S0wbDtYMEaPhc8s240', attr='<a href="http://jawg.io" title="Tiles Courtesy of Jawg Maps" target="_blank">&copy; <b>Jawg</b>Maps</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', control_scale=True, max_bounds=True, prefer_canvas=True, zoom_start=10)
for ind, row in meta[meta.index.isin(stations)].iterrows():
    icon = folium.features.CustomIcon(f"./data/icons/north.png", icon_size=(15, 15))
    style = 'font-family: Impact, Haettenschweiler, "Franklin Gothic Bold", Charcoal, "Helvetica Inserat", "Bitstream Vera Sans Bold", "Arial Black", "sans serif";'
    html = f'''<body style="{style}"><div align="center", style="background-color: #BAD6FF; font-family: Arial">''' + row.to_frame().transpose()[['ID', 'Fwy', 'Length', 'Type', 'Lanes']].transpose().to_html(justify="center", header=False, index=True, index_names=False, col_space=300, classes="table-condensed table-responsive table-success") + '</div></body>' 
    popup = folium.Popup(html, max_width=300)                
    folium.Marker(row[['Latitude', 'Longitude']].values.tolist(), icon=icon, popup=popup).add_to(m)
m

# EDA for Speed Data

In [82]:
import pickle

In [ ]:
with open('./data/loaded/meta.dat', 'rb') as f:
    speed_df = pickle.load(f)
meta